In [221]:
import pygame,sys
from pygame.locals import *
from random import randint

In [222]:
#colors
WHITE = (255, 255, 255)
BLACK=(0,0,0)

In [223]:
def desloc(x,y,ori):
    des=1
    if(ori=='r'):
        return (x+des,y)
    elif(ori=='l'):
        return (x-des,y)
    elif(ori=='u'):
        return (x,y-des)
    elif (ori=='d'):
        return (x,y+des)

In [224]:
def oriNumb(ori):
    
    if(ori=='r'):
        return 0
    elif(ori=='d'):
        return 1
    elif(ori=='l'):
        return 2
    elif(ori=='u'):
        return 3

In [225]:
class food(pygame.sprite.Sprite):
    
    def __init__(self,x,y,img):
        
        pygame.sprite.Sprite.__init__(self)
        self.image=img
        self.rect=self.image.get_rect()
        self.rect.x=x
        self.rect.y=y

In [226]:
#head
class head(pygame.sprite.Sprite):
    
    def __init__(self,x,y,img):
        
        pygame.sprite.Sprite.__init__(self)
        self.image=img
        self.rect=pygame.rect.Rect(x,y,int(img.get_width()*0.75),int(img.get_height()+0.75))
        self.orientation=''
        self.turns=[]
    
    def get_render_rect(self):
        
        return ((self.rect.x-int(self.image.get_width()*0.25)),(self.rect.y-int(self.image.get_height()*0.25)))

In [227]:
#body
class body(pygame.sprite.Sprite):
    def __init__(self,x,y,ori,img):
        
        pygame.sprite.Sprite.__init__(self)
        self.image=img
        self.rect=pygame.rect.Rect(x,y,int(img.get_width()*0.75),int(img.get_height()*0.75))
        self.orientation=ori
        self.turns=[]
    
    def move(self):
        
        if(len(self.turns)==0):
            self.rect.x,self.rect.y=desloc(self.rect.x,self.rect.y,self.orientation)
            
        else:
            turnx,turny,turno= self.turns[0]
            virtualx,virtualy=desloc(self.rect.x,self.rect.y,self.orientation)
            
            if(((self.orientation=='r' and virtualx>turnx) or (self.orientation=='l' and virtualx<turnx) 
               or (self.orientation=='u' and virtualy<turny) or (self.orientation=='d' and virtualy>turny))
              and self.orientation!=turno):
                
                self.rect.x,self.rect.y=desloc(self.rect.x,self.rect.y,turno)
                self.orientation=turno
                self.turns.pop(0)
                
            else:
                self.rect.x,self.rect.y=desloc(self.rect.x,self.rect.y,self.orientation)

    def get_render_rect(self):
        
        return ((self.rect.x-int(self.image.get_width()*0.25)),(self.rect.y-int(self.image.get_height()*0.25)))
                

In [228]:
#Snake
class Snake:
    
    def __init__(self,x,y,headIm,ori='r'):
        
        self.head=head(x,y,headIm)
        self.head.orientation=ori
        self.blocks=[]
        self.orientation=ori   
        self.headImg=headIm
        self.bodyImg=pygame.Surface((0,0))
        self.walls=[]
        
    def move(self,ori):
        
        if(ori!=self.orientation):
            #turn happened
            turn=(self.head.rect.x,self.head.rect.y,ori)
            for block in self.blocks:
                block.turns.append(turn)
            self.orientation=ori
            self.head.orientation=ori
   
        self.head.rect.x,self.head.rect.y=desloc(self.head.rect.x,self.head.rect.y,ori)
        
        for b in self.blocks:
            b.move()

    def createWalls(self,dim):
        
        r1=pygame.rect.Rect(0,-1,dim,1)
        r2=pygame.rect.Rect(-1,0,1,dim)
        r3=pygame.rect.Rect(dim,0,1,dim)
        r4=pygame.rect.Rect(0,dim,dim,1)
        self.walls.append(r1)
        self.walls.append(r2)
        self.walls.append(r3)
        self.walls.append(r4)
    
    def isAlive(self):
        
        head_rect=self.head.rect
        
        for bloc in self.blocks:
            if(head_rect.colliderect(bloc.rect)):
                return False
        
        for wall in self.walls:
            if(head_rect.colliderect(wall)):
                return False
        return True
    
    def calcBodyPosition(self,bloc):
        
        ori=bloc.orientation
        
        #values
        spacing=int(0.70*bloc.image.get_width())
        dist=spacing+bloc.image.get_width()
        
        #virtual values
        virtualx,virtualy=0,0
        
        if(ori=='r'):
            virtualx,virtualy=(bloc.rect.x-dist),(bloc.rect.y)

        elif(ori=='l'):
            virtualx,virtualy=(bloc.rect.x+dist),(bloc.rect.y)

        elif(ori=='u'):
            virtualx,virtualy=(bloc.rect.x),(bloc.rect.y+dist)

        elif(ori=='d'):
            virtualx,virtualy=(bloc.rect.x),(bloc.rect.y-dist)
        
        return (virtualx,virtualy)
    
    def addBlock(self):
        
        lastbloc=self.head
        
        if(len(self.blocks)>0):
            lastbloc=self.blocks[-1]
        bx,by=self.calcBodyPosition(lastbloc)
        nbody= body(bx,by,lastbloc.orientation,lastbloc.image)
        nbody.turns=list(lastbloc.turns)
        self.blocks.append(nbody)
        
    def eaten(self,food):
        
        if(self.head.rect.colliderect(food.rect)):
            return True
        return False
    
    
    def obstDist(self,ori):
        
        dx,dy=0,0
        if(ori==0):
            dx=1
        elif(ori==2):
            dx=-1
        elif(ori==3):
            dy=-1
        elif(ori==1):
            dy=1
        
        virtualx,virtualy=self.head.rect.x,self.head.rect.y
        flag=True
        while(flag):
            virtualx+=dx
            virtualy+=dy
            for bloc in self.blocks:
                bx,by=bloc.get_render_rect()
                sp=int(bloc.image.get_width()*0.7)
                nrect=pygame.rect.Rect(bx,by,(bloc.image.get_width()+sp),(bloc.image.get_height()+sp))
                if(nrect.collidepoint(virtualx,virtualy)):
                    flag=False
                    break
            
            if(flag):    
                for wall in self.walls:
                    if(wall.collidepoint(virtualx,virtualy)):
                        flag=False
                        break
        if(ori==0 or ori==2):
            return abs((self.head.rect.x-virtualx))
        elif(ori==1 or ori==3):
            return abs((self.head.rect.y-virtualy))

    
    def getAtr(self):
        
        nori=oriNumb(self.orientation)
        return [self.obstDist((nori+3)%4),self.obstDist(nori),self.obstDist((nori+1)%4)]

In [229]:
def renderSnake(display,snake):
    
    headImg=snake.headImg
    bodyImg=snake.bodyImg
    
    #add head
    display.blit(headImg,snake.head.get_render_rect())
    
    #add body
    for body in snake.blocks:
        display.blit(bodyImg,body.get_render_rect()) 
        

In [230]:
def renderFood(display,food):
    
    display.blit(food.image,food.rect)

In [231]:
def newFood(foodImg,dim):
    
    fos=5
    return food(randint(fos,(dim-foodImg.get_width()-fos)),randint(fos,(dim-foodImg.get_height()-fos)),foodImg)

In [232]:
#init
pygame.init()
dim=300
displaysurf=pygame.display.set_mode((dim,dim))
pygame.display.set_caption('Snaike')
fpsClock = pygame.time.Clock()
FPS = 120


#emilia variables
emiliaImg = pygame.image.load('blue.png')
emiliaSnake = Snake(int(dim/2),int(dim/2),emiliaImg,ori='d')
emiliaSnake.createWalls(dim)

#subaro
subaroImg=pygame.image.load('blue.png')
emiliaSnake.bodyImg=subaroImg

#game over
gameoverImg=pygame.image.load('gameover.jpg')

#food
foodImg=pygame.image.load('hamb.png')
foood=newFood(foodImg,dim)

#emiliaSnake.addBlock()
for _ in range(15):
    emiliaSnake.addBlock()
orientation=emiliaSnake.orientation
gameOverFlag=False

#game loop
while True:   
    displaysurf.fill(WHITE)
    
    for event in pygame.event.get():
        if event.type== QUIT:
            pygame.quit()
            sys.exit()
        elif event.type==KEYDOWN:
            if event.key==K_UP and orientation!='d':
                orientation='u'
            elif event.key==K_DOWN and orientation!='u':
                orientation='d'
            elif event.key==K_RIGHT and orientation!='l':
                orientation='r'
            elif event.key==K_LEFT and orientation!='r':
                orientation='l'
            elif event.key==K_f:
                emiliaSnake = Snake(int(dim/2),int(dim/2),emiliaImg)
                emiliaSnake.createWalls(dim)
                emiliaSnake.bodyImg=subaroImg
                gameOverFlag=False
            elif event.key==K_n:
                pygame.quit()
                sys.exit()
    
    #emilia motion
    emiliaSnake.move(orientation)
    
    if(not emiliaSnake.isAlive()):
        gameOverFlag=True
        
        displaysurf.blit(gameoverImg, (int((dim-gameoverImg.get_width())/2),(int((dim-gameoverImg.get_height())/2)-50)))
        pygame.display.update()

    if(emiliaSnake.eaten(foood)):
        
        foood=newFood(foodImg,dim)
        emiliaSnake.addBlock()
    
    if(not gameOverFlag):
        renderSnake(displaysurf,emiliaSnake)
        renderFood(displaysurf,foood)
        print(emiliaSnake.getAtr())
        pygame.display.update()
        
    fpsClock.tick(FPS)

            

[150, 149, 151]
[150, 148, 151]
[150, 147, 151]
[150, 146, 151]
[150, 145, 151]
[150, 144, 151]
[150, 143, 151]
[150, 142, 151]
[150, 141, 151]
[150, 140, 151]
[150, 139, 151]
[150, 138, 151]
[150, 137, 151]
[150, 136, 151]
[150, 135, 151]
[150, 134, 151]
[150, 133, 151]
[150, 132, 151]
[150, 131, 151]
[150, 130, 151]
[150, 129, 151]
[150, 128, 151]
[150, 127, 151]
[150, 126, 151]
[150, 125, 151]
[150, 124, 151]
[150, 123, 151]
[150, 122, 151]
[150, 121, 151]
[150, 120, 151]
[150, 119, 151]
[150, 118, 151]
[150, 117, 151]
[150, 116, 151]
[150, 115, 151]
[150, 114, 151]
[150, 113, 151]
[150, 112, 151]
[150, 111, 151]
[150, 110, 151]
[150, 109, 151]
[150, 108, 151]
[150, 107, 151]
[150, 106, 151]
[150, 105, 151]
[150, 104, 151]
[150, 103, 151]
[150, 102, 151]
[150, 101, 151]
[150, 100, 151]
[150, 99, 151]
[150, 98, 151]
[150, 97, 151]
[150, 96, 151]
[150, 95, 151]
[150, 94, 151]
[150, 93, 151]
[150, 92, 151]
[150, 91, 151]
[150, 90, 151]
[150, 89, 151]
[150, 88, 151]
[150, 87, 151]
[150,

SystemExit: 

C:\Users\Fernando\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
